<a href="https://colab.research.google.com/github/fbeilstein/machine_learning/blob/master/python_refresher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.assoc_laguerre.html

from scipy.special import assoc_laguerre
from scipy.special import binom
import numpy as np

K = 3

def x_0(alpha, k):
  return np.sqrt(2) * alpha * np.cos(k * np.pi / K)

def E(sigma, alpha, s, k, n):
  f = sigma**2 / (s + sigma**2)
  y = - x_0(alpha, k)**2 / (s + sigma**2)
  return s**n * f**(n + 1/2) * np.exp(y) * assoc_laguerre(y * sigma**2 / s, n, -1/2)

def E_tot(sigma, n):
  f = sigma**2 / (1 + sigma**2)
  return 0.5 * (-1)**(n+1) * f**(n+0.5) * (binom(-1/2, n) - 2 * f * binom(-3/2, n))

def E_simpl(r, phi, sigma, k, n):
  return (r**2)**n * (np.cos(np.pi * k / K + phi))**(2*n) * np.exp(- r**2 * np.cos(np.pi * k / K + phi)**2 / sigma**2)

print(x_0(1, 1))
print(E(0.2, 1, 1, 1, 1))

0.7071067811865477
0.0024216143114033704


In [0]:
from scipy.optimize import minimize

K = 3
s = 0
sigma = 1

C_n = np.array([0.0] * 2, dtype=float)
alpha = 0.0

def func_to_minimize(params):
  C_n = np.array([1.0] + params.tolist())
  number_of_C = C_n.shape[0]
  E_totals = np.array([E_tot(sigma, n) for n in range(number_of_C)])
  LHS = K * np.abs(np.dot(E_totals, C_n))

  def min_in_min(a):
    #Es = np.array([[E(sigma, a[0], s, k, n) for k in range(K)] for n in range(number_of_C)])
    Es = np.array([[E_simpl(a[0], a[1], sigma, k, n) for k in range(K)] for n in range(number_of_C)])
    RHS = np.abs(np.dot(np.sum(Es, axis=1), C_n))
    #print(a, RHS)
    return -RHS

  alpha_min = minimize(min_in_min, [0.1, 0.0], 
         method='Nelder-Mead', jac=None, hess=None,
         options={'maxiter':100}, callback=None)

  #Es = np.array([[E(sigma, alpha_min['x'][0], s, k, n) for k in range(K)] for n in range(number_of_C)])
  Es = np.array([[E_simpl(alpha_min['x'][0], alpha_min['x'][1], sigma, k, n) for k in range(K)] for n in range(number_of_C)])
  #print(alpha_min['x'][0], alpha_min['x'][1], Es)


  RHS = np.abs(np.dot(np.sum(Es, axis=1), C_n))
  return - LHS / (RHS + 1E-5)

minimize(func_to_minimize, C_n, 
         method='Nelder-Mead', jac=None, hess=None,
         options={'maxiter':10}, callback=None)

In [0]:
exec('print("hello world")')

hello world


In [0]:
directions = [[1,0], [0,1], [-1,0], [0,-1]]
l = 20
cur_x = 0
cur_y = 0
cur_dir = 0
out = [['___' for _ in range(l+1)] for _ in range(l+1)]
it = 0
while l > 0:
  for i in range(l):
    cur_x += directions[cur_dir][0]
    cur_y += directions[cur_dir][1]
    s = str(it)
    while len(s) < 3:
      s = ' ' + s      
    out[cur_x][cur_y] = s
    it += 1
  l -= 1
  cur_dir = (cur_dir + 1) % 4

for line in out:
  print(line)

['___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___']
['  0', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___']
['  1', '___', ' 73', ' 72', ' 71', ' 70', ' 69', ' 68', ' 67', ' 66', ' 65', ' 64', ' 63', ' 62', ' 61', ' 60', ' 59', ' 58', ' 57', ' 56', '___']
['  2', '___', ' 74', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', ' 55', '___']
['  3', '___', ' 75', '___', '131', '130', '129', '128', '127', '126', '125', '124', '123', '122', '121', '120', '119', '118', '___', ' 54', '___']
['  4', '___', ' 76', '___', '132', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '___', '117', '___', ' 53', '___']
['  5', '___', ' 77', '___', '133', '___', '173', '172', '171', '170', '169', '168', '167', '166', '165', '164',

In [0]:
x = 2
y = 3
x + y

5

In [0]:
if x > y:
  print("greater")
else:
  print("not greater")

not greater


In [0]:
for i in range(5):
  print(i, end=',')

0,1,2,3,4,

In [0]:
L = [2,3,5,7,11,13,17]
L

[2, 3, 5, 7, 11, 13, 17]

In [0]:
for prime in L:
  print(prime, end=',')

2,3,5,7,11,13,17,

In [0]:
p = 5
if p in L:
  print(p, ' is prime')

5  is prime


In [0]:
L_extended = L + [19, 23]
L_extended

[2, 3, 5, 7, 11, 13, 17, 19, 23]

In [0]:
L_extended[2:5]

[5, 7, 11]

In [0]:
L_extended[:-3]

[2, 3, 5, 7, 11, 13]

In [0]:
while L:
  L = L[1:]
  print(L)

[3, 5, 7, 11, 13, 17]
[5, 7, 11, 13, 17]
[7, 11, 13, 17]
[11, 13, 17]
[13, 17]
[17]
[]


In [0]:
def func(arg1, arg2):
  return arg1 + arg2

x = func(3, 4)
x

7

In [0]:
[i**2 for i in range(5)]

[0, 1, 4, 9, 16]

In [0]:
def func(x):
  x[0] += 175  

L = [i**2 for i in range(5)]
print(L)
func(L)
print(L)

[0, 1, 4, 9, 16]
[175, 1, 4, 9, 16]


In [0]:
def func(x):
  y = x.copy()
  y[0] += 175

L = [i**2 for i in range(5)]
print(L)
func(L)
print(L)

[0, 1, 4, 9, 16]
[0, 1, 4, 9, 16]


In [0]:
class SomeClass:

  def __init__(self, arg1, arg2):
    self._sum = arg1 + arg2
    self._diff = arg1 - arg2

  def printer(self):
    print(self._sum, self._diff)


obj = SomeClass(3, 4)
obj.printer()

7 -1


In [0]:
class SomeOtherClass:

  def __init__(self, arg1, arg2):
    self._sum = arg1 + arg2
    self._diff = arg1 - arg2

  def update_sum(self, arg):
    self._sum = arg

  def printer(self):
    print(self._sum, self._diff)


obj = SomeOtherClass(3, 4)
obj.printer()
obj.update_sum(77)
obj.printer()

7 -1
77 -1


**How to debug**

* Don't hesitate to change or comment out code for debug
* Print intermediate results if you are not sure what function does
* Separate logical blocks as functions
* Write tests, test your functions on inputs you can deduce the output for


**Code style**

* Use meaningful names
* Use single style (CamelStyle, underscore_style)
* Use some conventions for class-function-variable names
* Isolate logical blocks as functions, consider creating classes to save states
* Use first underscore convention for private variables